#### 加载训练集与验证集，并使用keras中的数据增强功能

In [1]:
from keras.preprocessing.image import ImageDataGenerator

# train和test文件夹下都有四个文件夹，名字分别为0、90、180、270，各自里面存放着对应角度的图片
train_dir = 'C:/work/images/receipts/train/'
test_dir = 'C:/work/images/receipts/test/'

# train_datagen =  ImageDataGenerator()
# test_datagen = ImageDataGenerator()

train_datagen =  ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    channel_shift_range=0.1,
    brightness_range=(0.6, 1)
)

test_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    channel_shift_range=0.1,
    brightness_range=(0.6, 1)
)

train_generator = train_datagen.flow_from_directory(
  train_dir,
  target_size=(224, 224),
  batch_size=32,
)
validation_generator = test_datagen.flow_from_directory(
  test_dir,
  target_size=(224, 224),
  batch_size=32,
)

Using TensorFlow backend.


Found 10120 images belonging to 4 classes.
Found 1200 images belonging to 4 classes.


#### 定义模型，这里采用迁移学习方式，使用keras中的VGG19预训练模型

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras import backend as K

path = "C:/Users/E101080/.keras/models/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"

base_model = VGG19(weights=path, include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Flatten(name='flatten')(base_model.output)

model = Dense(256, activation='relu', name='fc1')(model)
model = Dense(512, activation='relu', name='fc2')(model)
model = Dropout(0.5)(model)

# 只定了四个方向0、90、180、270度
model = Dense(4, activation='softmax')(model)

model = Model(inputs=base_model.input, outputs=model, name='vgg19')

# 打印模型结构，包括所需要的参数
# model.summary()

# 编译模型
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#### 开始训练

In [8]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

model.fit_generator(
  train_generator,
  steps_per_epoch=320,
  epochs=6,
  validation_data=validation_generator)

Epoch 1/6
320/320 [==============================] - 1895s 6s/step - loss: 0.3925 - accuracy: 0.8885 - val_loss: 1.0238 - val_accuracy: 0.9058
Epoch 2/6
320/320 [==============================] - 1799s 6s/step - loss: 0.3660 - accuracy: 0.8948 - val_loss: 1.7547 - val_accuracy: 0.9217
Epoch 3/6
320/320 [==============================] - 1778s 6s/step - loss: 0.3168 - accuracy: 0.9115 - val_loss: 0.0837 - val_accuracy: 0.9358
Epoch 4/6
320/320 [==============================] - 1780s 6s/step - loss: 0.3002 - accuracy: 0.9150 - val_loss: 0.0053 - val_accuracy: 0.9325
Epoch 5/6
320/320 [==============================] - 1779s 6s/step - loss: 0.2665 - accuracy: 0.9207 - val_loss: 0.2951 - val_accuracy: 0.9142
Epoch 6/6
320/320 [==============================] - 1773s 6s/step - loss: 0.2665 - accuracy: 0.9229 - val_loss: 0.1965 - val_accuracy: 0.9275


In [9]:
# 保存模型
model.save('vgg19-256-512.h5')

#### 预测

In [ ]:
# 查看label
train_generator.class_indices

In [ ]:
import numpy as np

test_image_path = r'C:\Users\E101080\Downloads\pic\consult + med printed (3).TIF'

valid_image = image.load_img(test_image_path, target_size=(150, 150))

image_arr = image.img_to_array(valid_image)

np.argmax(model.predict(np.expand_dims(image_arr, 0)),axis=1)

#### 加载保存后的模型并进行预测

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
test_image_path = r'C:\Users\E101080\Downloads\pic\consult + med printed (3).TIF'

my_model = load_model('vgg19-256-512.h5')

valid_image = image.load_img(test_image_path, target_size=(150, 150))
image_arr = image.img_to_array(valid_image)

np.argmax(my_model.predict(np.expand_dims(image_arr, 0)),axis=1)

#### 重新加载模型进行微调

In [5]:
from keras.models import load_model
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# my_model = load_model('vgg19-256-512-92.h5')

my_model.fit_generator(
  train_generator,
  steps_per_epoch=100,
  epochs=2,
  validation_data=validation_generator)

Epoch 1/2
100/100 [==============================] - 707s 7s/step - loss: 0.1277 - accuracy: 0.9663 - val_loss: 0.1412 - val_accuracy: 0.9517
Epoch 2/2
100/100 [==============================] - 753s 8s/step - loss: 0.1412 - accuracy: 0.9622 - val_loss: 0.1374 - val_accuracy: 0.9425


In [6]:
# save model
my_model.save('vgg19-256-512-94.h5')